# LangChain Expression Language (LCEL)
* LCEL is ideal when you want to build and orchestrate multi-step workflows for LLM applications in a clear and maintainable way.
```
      Prompt + Model + Output Parsing (Simple Chain)

Option 1: Direct OpenAI SDK
* Official OpenAI SDK, typically used when you're not using LangChain

In [ ]:
import os
import openai as OpenAI

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

In [ ]:

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

response = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages= [{'role': "user", "content": "Tell me a joke about Monkey"}]
)
print(response.choices[0].message.content)

Option 2: LangChain Wrapper (ChatOpenAI())
* Use this only if you're working within the LangChain framework, which abstracts models, prompts, memory, chains, etc

In [ ]:
!pip install --force-reinstall langchain langchain-core
##!pip install "pydantic>=2.7.4"

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain using Invoke Method

In [ ]:
prompt= ChatPromptTemplate.from_template("Tell me a joke about {topic}")

model = ChatOpenAI()

output_parser= StrOutputParser()

In [ ]:
chain= prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "monkey"})

## More complex chain

RunnableMap is like a pre-processor step in a LangChain pipeline.

It maps the inputs into the format that the next block (prompt) expects.
In this case, you’re preparing a dictionary:

``` 
{"context": [...relevant docs...], "question": "your original question"} 
```

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
!pip install docarray

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Practice makes perfect", "think before you speak"],
        embedding=OpenAIEmbeddings())

retriever= vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents("what makes perfect?")

In [ ]:
retriever.get_relevant_documents("what to do before speaking")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain= RunnableMap({ "context" : lambda x: retriever.get_relevant_documents(x['question']),
             "question" : lambda x: x['question']}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "what makes perfect?"})

## Bind Method

bind() is used to create a new instance of the model with certain parameters fixed.

Think of it like "attaching" or "locking in" a configuration (e.g., function calling schema)

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {"type": "object","properties": {"airport_code": {"type": "string",
                    "description": "The airport code to get the weather for"},},"required": ["airport_code"]}
    }
  ]

In [ ]:
functions[0]['parameters']['properties']['airport_code']

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}")
])
model = ChatOpenAI(temperature = 0).bind(functions= functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [ ]:
model = model.bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

## Fallbacks
* Fallbacks increase robustness if one model/pipeline fails to return usable results.

In [ ]:
!pip install -U langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
import json

In [ ]:
simple_model = ChatOpenAI(
    temperature = 0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo"
)
simple_chain = simple_model | json.loads

In [ ]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [ ]:
simple_model.invoke(challenge)

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [ ]:
simple_chain.invoke(challenge)

In [51]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [52]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [53]:
final_chain = simple_chain.with_fallbacks([chain])

In [54]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

## Interface
In LangChain (or other similar frameworks), the .invoke, .batch, and .stream methods are used to interact with chains or models in different ways:

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"}) # a single input to the chain or model and returns the output.

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}]) ## Sends a list of inputs

In [ ]:
for t in chain.stream({"topic": "bears"}): 
    print(t)  # Good for UIs or applications where partial results are needed in real-time.

In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response

# RAG (Retrieval-Augmented Generation)

In [75]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser
from langchain.schema.runnable import RunnableMap
from langchain.output_parsers import ResponseSchema
from langchain.retrievers import WikipediaRetriever

In [63]:
!pip install wikipedia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=df4b127df26109d7c8f46b74d03cf36b5fb3d0147b14dd4d76ece4b10a486616
  Stored in directory: c:\users\oadsa\appdata\local\pip\cache\wheels\8f\ab\cb\45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\oadsa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema.runnable import RunnableMap
from langchain.retrievers import WikipediaRetriever

# First Define response schema
response_schemas = [
    ResponseSchema(name="answer", description="Answer to the user's question")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Second step Get format instructions
format_instructions = output_parser.get_format_instructions()

# Third step Create Prompt with instructions
prompt = ChatPromptTemplate.from_template("""
You must answer the question using the context provided.

Return your answer as a JSON object with this format:
{format_instructions}

Context:
{context}

Question:
{question}
""")
final_prompt = prompt.partial(format_instructions=format_instructions)

# Fourth step DevelofModel and retriever
model = ChatOpenAI(temperature=0)
retriever = WikipediaRetriever()

# Finally create Chain using runnablemap
chain = (
    RunnableMap({
        "context": lambda x: retriever.get_relevant_documents(x["question"]),
        "question": lambda x: x["question"]
    })
    | final_prompt
    | model
    | output_parser
)

# Run it
result = chain.invoke({"question": "What is LangChain?"})
print(result)


{'answer': 'LangChain is one of the generative AI frameworks that Milvus can integrate with for monitoring and alerts.'}


```
This LangChain pipeline uses a WikipediaRetriever to fetch context based on a user’s question and then generates a structured JSON answer using a prompt template and ChatOpenAI. The StructuredOutputParser enforces a specific output format defined by a response schema.
```